In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout

In [4]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [5]:
# установка параметров нейросети
batch_size = 512
num_classes = 10
epochs = 10
data_augmentation = False
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'тренировочные примеры')
print(X_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


X_train = X_train / 255.0
X_test = X_test / 255.0

X_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [6]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)       

In [7]:
data_augmentation = True

In [8]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit_generator
    model.fit_generator(train_gen,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=6)

Использование data augmentation


C:\Users\Андрей\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
98/98 [==============================] - 46s 457ms/step - loss: 2.0329 - accuracy: 0.2418 - val_loss: 1.5767 - val_accuracy: 0.4273
Epoch 2/10
98/98 [==============================] - 42s 416ms/step - loss: 1.5906 - accuracy: 0.4240 - val_loss: 1.3755 - val_accuracy: 0.4984
Epoch 3/10
98/98 [==============================] - 42s 417ms/step - loss: 1.4550 - accuracy: 0.4778 - val_loss: 1.2740 - val_accuracy: 0.5372
Epoch 4/10
98/98 [==============================] - 42s 415ms/step - loss: 1.3628 - accuracy: 0.5109 - val_loss: 1.1333 - val_accuracy: 0.5975
Epoch 5/10
98/98 [==============================] - 41s 408ms/step - loss: 1.2739 - accuracy: 0.5460 - val_loss: 1.1752 - val_accuracy: 0.5784
Epoch 6/10
98/98 [==============================] - 42s 417ms/step - loss: 1.2199 - accuracy: 0.5678 - val_loss: 1.0237 - val_accuracy: 0.6368
Epoch 7/10
98/98 [==============================] - 42s 418ms/step - loss: 1.1638 - accuracy: 0.5900 - val_loss: 0.9540 - val_accuracy: 0.6578

### оценить рост точности при увеличении ширины сети (больше фильтров)

In [9]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling

model.add(Conv2D(32, (2, 2), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (2, 2), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (2, 2), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))




# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        416       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 31, 31, 32)        4128      
_________________________________________________________________
activation_7 (Activation)    (None, 31, 31, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 64)       

In [10]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit_generator
    model.fit_generator(train_gen,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=6)

Использование data augmentation
Epoch 1/10
98/98 [==============================] - 39s 379ms/step - loss: 2.1188 - accuracy: 0.1987 - val_loss: 1.7277 - val_accuracy: 0.3662
Epoch 2/10
98/98 [==============================] - 38s 376ms/step - loss: 1.7429 - accuracy: 0.3602 - val_loss: 1.5044 - val_accuracy: 0.4396
Epoch 3/10
98/98 [==============================] - 39s 380ms/step - loss: 1.5833 - accuracy: 0.4161 - val_loss: 1.3676 - val_accuracy: 0.4946
Epoch 4/10
98/98 [==============================] - 39s 389ms/step - loss: 1.4771 - accuracy: 0.4640 - val_loss: 1.2826 - val_accuracy: 0.5300
Epoch 5/10
98/98 [==============================] - 40s 389ms/step - loss: 1.3883 - accuracy: 0.5006 - val_loss: 1.3541 - val_accuracy: 0.5156
Epoch 6/10
98/98 [==============================] - 38s 374ms/step - loss: 1.3359 - accuracy: 0.5171 - val_loss: 1.2064 - val_accuracy: 0.5639
Epoch 7/10
98/98 [==============================] - 38s 375ms/step - loss: 1.2755 - accuracy: 0.5407 - val_los

оценить рост точности при увеличении глубины сети (больше слоев)

In [11]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_14 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_15 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 15, 15, 64)       

In [12]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit_generator
    model.fit_generator(train_gen,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=6)

Использование data augmentation
Epoch 1/10
98/98 [==============================] - 43s 427ms/step - loss: 2.1235 - accuracy: 0.1885 - val_loss: 1.6260 - val_accuracy: 0.4018
Epoch 2/10
98/98 [==============================] - 43s 427ms/step - loss: 1.6365 - accuracy: 0.3911 - val_loss: 1.3713 - val_accuracy: 0.4975
Epoch 3/10
98/98 [==============================] - 43s 433ms/step - loss: 1.4668 - accuracy: 0.4607 - val_loss: 1.3152 - val_accuracy: 0.5283
Epoch 4/10
98/98 [==============================] - 43s 433ms/step - loss: 1.3534 - accuracy: 0.5128 - val_loss: 1.1993 - val_accuracy: 0.5648
Epoch 5/10
98/98 [==============================] - 42s 422ms/step - loss: 1.2663 - accuracy: 0.5456 - val_loss: 1.0500 - val_accuracy: 0.6232
Epoch 6/10
98/98 [==============================] - 43s 425ms/step - loss: 1.1890 - accuracy: 0.5744 - val_loss: 1.0979 - val_accuracy: 0.6075
Epoch 7/10
98/98 [==============================] - 43s 425ms/step - loss: 1.1448 - accuracy: 0.5893 - val_los

In [13]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (2, 2), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_22 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_23 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 15, 15, 64)       

In [14]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit_generator
    model.fit_generator(train_gen,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=6)

Использование data augmentation
Epoch 1/10
98/98 [==============================] - 47s 459ms/step - loss: 2.1991 - accuracy: 0.1566 - val_loss: 1.8455 - val_accuracy: 0.3043
Epoch 2/10
98/98 [==============================] - 46s 459ms/step - loss: 1.8382 - accuracy: 0.2988 - val_loss: 1.5882 - val_accuracy: 0.3883
Epoch 3/10
98/98 [==============================] - 45s 448ms/step - loss: 1.6334 - accuracy: 0.3915 - val_loss: 1.3965 - val_accuracy: 0.4814
Epoch 4/10
98/98 [==============================] - 45s 453ms/step - loss: 1.4829 - accuracy: 0.4537 - val_loss: 1.2987 - val_accuracy: 0.5256
Epoch 5/10
98/98 [==============================] - 46s 455ms/step - loss: 1.4135 - accuracy: 0.4863 - val_loss: 1.2868 - val_accuracy: 0.5336
Epoch 6/10
98/98 [==============================] - 45s 453ms/step - loss: 1.3489 - accuracy: 0.5108 - val_loss: 1.2644 - val_accuracy: 0.5502
Epoch 7/10
98/98 [==============================] - 46s 459ms/step - loss: 1.2926 - accuracy: 0.5343 - val_los

In [38]:
(train_images,train_labels),(test_images,test_labels)=cifar10.load_data()

train_images = (train_images / 127) - 1
test_images = (test_images / 127) - 1

y_t = to_categorical(train_labels)
y_tt = to_categorical(test_labels)

In [54]:
model = Sequential()

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

#model.summary()
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [55]:
if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=10, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0.2, 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)


    train_gen = datagen.flow(X_train, 
                             y_train,
                             batch_size=batch_size)

    # запуск data augmentation через fit_generator
    model.fit_generator(train_gen,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=6)

Использование data augmentation


C:\Users\Андрей\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
98/98 [==============================] - 19s 179ms/step - loss: 2.1699 - accuracy: 0.1927 - val_loss: 1.8730 - val_accuracy: 0.3346
Epoch 2/10
98/98 [==============================] - 18s 177ms/step - loss: 1.9519 - accuracy: 0.2823 - val_loss: 1.8167 - val_accuracy: 0.3523
Epoch 3/10
98/98 [==============================] - 19s 183ms/step - loss: 1.9105 - accuracy: 0.3016 - val_loss: 1.7564 - val_accuracy: 0.3659
Epoch 4/10
98/98 [==============================] - 19s 180ms/step - loss: 1.8798 - accuracy: 0.3101 - val_loss: 1.7291 - val_accuracy: 0.3838
Epoch 5/10
98/98 [==============================] - 19s 175ms/step - loss: 1.8582 - accuracy: 0.3198 - val_loss: 1.7491 - val_accuracy: 0.3705
Epoch 6/10
98/98 [==============================] - 19s 175ms/step - loss: 1.8453 - accuracy: 0.3236 - val_loss: 1.6841 - val_accuracy: 0.4031
Epoch 7/10
98/98 [==============================] - 19s 176ms/step - loss: 1.8216 - accuracy: 0.3369 - val_loss: 1.6830 - val_accuracy: 0.4101

* Во время обучения моделей видно что не хватает кол-ва эпох
* лучшый результат показала сеть в которой было два светчных слоя и более глубокие полносвязные слои
* немного хуже та модель где были больше фильтров
* для более лучшего результата надо будет подбирать кол-во фильтров и делать более глубокие полносвязные слои, и использовать early stop
* Использование только лишь полносвязных слоев дает самый плохой результат